In [1]:
# Load the gams extension
%load_ext gams_magic

# Support Vector Machines

Support vector machines are a popular method for classification
within the machine learning community.  Essentially, a linear
  classifier $f$ is generated as $f(x) = w'x + \gamma$ such that if
  $f(x) > 0$ then $x$ is in class $P_+$ whereas if $f(x) < 0$ then $x
  \in P_-$.
  
Once you have the classifier, you can use it with new data $x$ to predict if
$x \in P_+$ or $x \in P_i$ just by evaluating $f(x)$.

In this exercise, you should use the data that is provided in the abalone.gdx file,
and you should attempt to predict whether the ``number of rings'' is greater than 10 or not.  The data file provides 4177 samples, with the following features (including the 
number of rings):

|Name           |Data Type      |Meas.  |Description                |
|---------------|---------------|-------|---------------------------|
|Sex            |nominal        |       |M, F, and I (infant)       |
|Length         |continuous     |mm     |Longest shell measurement  |
|Diameter       |continuous     |mm     |perpendicular to length    |
|Height         |continuous     |mm     |with meat in shell         |
|Whole weight   |continuous     |grams  |whole abalone              |
|Shucked weight |continuous     |grams  |weight of meat             |
|Viscera weight |continuous     |grams  |gut weight (after bleeding)|
|Shell weight   |continuous     |grams  |after being dried          |
|Rings          |integer        |       |+1.5 gives the age in years|
 
Note that the first nominal value has been converted to 3 binary
values and can be treated as separate data for this assignment.
The following gams code uses the set x to index the features, excluding the "rings" that is
stored as a label y indicating whether the number of rings is greater than 10 or not.

Instead of using all the data to generate the classifier $f$, we use a subset of the samples $i$ as training data, another set as tuning data to determine some tradeoff parameters and the remaining data as the testing data (to see how well we do).  You should pretend that you do not know the values of y for the testing data since normally you won't!

In [2]:
%%gams
set i(*), headr(*);
parameter Data(i,headr);
$gdxin abalone.gdx
$LOAD i
$LOAD headr
$LOAD Data
$gdxin

set  x(headr)  index of independent variables;
x(headr) = yes$(not sameas(headr,'Rings'));

parameter y(i);
y(i) = -1 + 2$(Data(i,'Rings') gt 10);

set train(i), tuning(i), test(i);
train(i) = yes$(ord(i) le 3000);
tuning(i) = yes$(ord(i) gt 3000 and ord(i) le 3500);
test(i) = yes$(ord(i) gt 3500);

To generate this classifier a quadratic optimization model is solved, using only the data in the training set:
$$
  \begin{array}{lc}
    \min_{w,\gamma,\psi} & \frac{1}{2} w'w + C \sum_i \psi_i \\
    \text{s.t.} & y_i [w'x_i + \gamma] \geq 1 - \psi_i, i=1,\ldots,m \\
    & \psi_i \geq 0
  \end{array}
$$
Here $m$ runs over the set of training examples, and $y_i$ is $1$ if $i \in P_+$ and $-1$ if $i \in P_-$.  $x_i$ are the values of the predictor variables.
Note that C is a tradeoff parameter between errors $\psi_i$ and a measure of generalization ability $w$ and we will choose an appropriate value of C.  Initially set $C=1$.
  
The dual problem of this quadratic program is:
$$
  \begin{array}{lc}
    \max_{\alpha} & \sum_i \alpha_i 
    - \frac{1}{2} \sum_{i,j} \alpha_i \alpha_j y_i y_j x_i' x_j \\
    \text{s.t.} & \sum_i y_i \alpha_i = 0 \\
    & C \geq \alpha_i \geq 0
  \end{array}
$$

### You should formulate both these problems in GAMS.  

Note that for the
  dual problem, you may want to introduce some intermediate variables 
$$
    v_k = \sum_i \alpha_i y_i x_{ik}
$$
  and express the objective function as
$$
  \sum_i \alpha_i 
  - \frac{1}{2} \sum_{k} v_k^2
$$
  where $k$ runs over the predictor features.  

In [4]:
%%gams
parameter C;
C = 1;

variables w(headr);
positive variable psi(i);
variables gamma;

variable ObjPrimal;

equation ObjPrimalDef;
ObjPrimalDef.. ObjPrimal =e= 1/2*sum(x(headr), w(headr)*w(headr)) + C*sum(train(i), psi(i) );

equation ConstraintPrimal;
ConstraintPrimal(i)$train(i).. y(i)*( sum(x(headr), w(headr)*Data(i,headr)) + gamma ) =g= 1 -  psi(i) ;

model Primal / ObjPrimalDef, ConstraintPrimal /;

positive variable alpha(i);
alpha.UP(i) = C;

variable v(headr);

equation vdef;
vdef(headr)$x(headr).. v(headr) =e= sum(train(i), alpha(i)*y(i)*Data(i,headr) ) ;

equation ConstraintDual;
ConstraintDual(headr)$x(headr).. sum(train(i), alpha(i)*y(i) ) =e= 0 ;

variable ObjDual;

equation ObjDualDef;
ObjDualDef.. ObjDual =e= sum(train(i), alpha(i) )  -  1/2 * sum(x(headr),  v(headr)*v(headr) ); 

model Dual / ObjDualDef, ConstraintDual, vdef /;


### Solve (the easiest one) using the qcp solver of your choice, and the use the multiplier information from this first solve to set the starting values for the second solve.  
Try to make the second solve take as little time as possible.   

In [5]:
%%gams
*# write gams code that solves one model here
*# then uses that solution to provide a (very good) starting point 
*# for the other model solve

solve Dual max ObjDual using QCP;
psi.L(i) = ConstraintDual.M(i);
solve Primal min ObjPrimal using QCP;


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),OptimalLocal (2),1534.0031,21,3011,QCP,CONOPT,0.64
1,Normal (1),OptimalLocal (2),1534.0031,3001,3012,QCP,CONOPT,2.391


Write code that experiments with multiple values of C and evaluate the number of errors 
you make on the tuning set.   Using a limited amount of computation (10 minutes at most), determine a good value for C and report it in the notebook.  

In [6]:
%%gams
*# write gams code here to calculate the value of C

parameter Ring_hat(i);
parameter y_hat(i);
parameter error(i);
parameter rate;

parameter Result(*,*);
parameter errors, C0, errors0, rate;

rate = 1;
errors0 = card(tuning);

*$ONTEXT
C = 1/2;
solve Dual max ObjDual using QCP;
psi.L(i)$tuning(i) = ConstraintDual.M(i);
solve Primal min ObjPrimal using QCP;
Ring_hat(i)$tuning(i) = sum(x(headr), w.L(headr)*Data(i,headr)) + gamma.L ;
y_hat(i) = -1 + 2$(Ring_hat(i) gt 1);
error(i) = 1;
error(i)$(y_hat(i) = y(i) ) = 0;
errors = sum(tuning(i),error(i));
if( errors0 > errors, C0=C; errors0 = errors );
rate = errors/card(tuning);
Result("C=1/2","errors") = errors;
Result("C=1/2","rate") = rate;

C = 1
solve Dual max ObjDual using QCP;
psi.L(i)$tuning(i) = ConstraintDual.M(i);
solve Primal min ObjPrimal using QCP;
Ring_hat(i)$tuning(i) = sum(x(headr), w.L(headr)*Data(i,headr)) + gamma.L ;
y_hat(i) = -1 + 2$(Ring_hat(i) gt 1);
error(i) = 1;
error(i)$(y_hat(i) = y(i) ) = 0;
errors = sum(tuning(i),error(i));
if( errors0 > errors, C0=C; errors0 = errors );
rate = errors/card(tuning);
Result("C=1","errors") = errors;
Result("C=1","rate") = rate;

C = 2
solve Dual max ObjDual using QCP;
psi.L(i)$tuning(i) = ConstraintDual.M(i);
solve Primal min ObjPrimal using QCP;
Ring_hat(i)$tuning(i) = sum(x(headr), w.L(headr)*Data(i,headr)) + gamma.L ;
y_hat(i) = -1 + 2$(Ring_hat(i) gt 1);
error(i) = 1;
error(i)$(y_hat(i) = y(i) ) = 0;
errors = sum(tuning(i),error(i));
if( errors0 > errors, C0=C; errors0 = errors );
rate = errors/card(tuning);
Result("C=2","errors") = errors;
Result("C=2","rate") = rate;

C = 3
solve Dual max ObjDual using QCP;
psi.L(i)$tuning(i) = ConstraintDual.M(i);
solve Primal min ObjPrimal using QCP;
Ring_hat(i)$tuning(i) = sum(x(headr), w.L(headr)*Data(i,headr)) + gamma.L ;
y_hat(i) = -1 + 2$(Ring_hat(i) gt 1);
error(i) = 1;
error(i)$(y_hat(i) = y(i) ) = 0;
errors = sum(tuning(i),error(i));
if( errors0 > errors, C0=C; errors0 = errors );
rate = errors/card(tuning);
Result("C=3","errors") = errors;
Result("C=3","rate") = rate;

C = 4
solve Dual max ObjDual using QCP;
psi.L(i)$tuning(i) = ConstraintDual.M(i);
solve Primal min ObjPrimal using QCP;
Ring_hat(i)$tuning(i) = sum(x(headr), w.L(headr)*Data(i,headr)) + gamma.L ;
y_hat(i) = -1 + 2$(Ring_hat(i) gt 1);
error(i) = 1;
error(i)$(y_hat(i) = y(i) ) = 0;
errors = sum(tuning(i),error(i));
if( errors0 > errors, C0=C; errors0 = errors );
rate = errors/card(tuning);
Result("C=4","errors") = errors;
Result("C=4","rate") = rate;

C = 5
solve Dual max ObjDual using QCP;
psi.L(i)$tuning(i) = ConstraintDual.M(i);
solve Primal min ObjPrimal using QCP;
Ring_hat(i)$tuning(i) = sum(x(headr), w.L(headr)*Data(i,headr)) + gamma.L ;
y_hat(i) = -1 + 2$(Ring_hat(i) gt 1);
error(i) = 1;
error(i)$(y_hat(i) = y(i) ) = 0;
errors = sum(tuning(i),error(i));
if( errors0 > errors, C0=C; errors0 = errors );
rate = errors/card(tuning);
Result("C=5","errors") = errors;
Result("C=5","rate") = rate;

C = 6
solve Dual max ObjDual using QCP;
psi.L(i)$tuning(i) = ConstraintDual.M(i);
solve Primal min ObjPrimal using QCP;
Ring_hat(i)$tuning(i) = sum(x(headr), w.L(headr)*Data(i,headr)) + gamma.L ;
y_hat(i) = -1 + 2$(Ring_hat(i) gt 1);
error(i) = 1;
error(i)$(y_hat(i) = y(i) ) = 0;
errors = sum(tuning(i),error(i));
if( errors0 > errors, C0=C; errors0 = errors );
rate = errors/card(tuning);
Result("C=6","errors") = errors;
Result("C=6","rate") = rate;

C = 7
solve Dual max ObjDual using QCP;
psi.L(i)$tuning(i) = ConstraintDual.M(i);
solve Primal min ObjPrimal using QCP;
Ring_hat(i)$tuning(i) = sum(x(headr), w.L(headr)*Data(i,headr)) + gamma.L ;
y_hat(i) = -1 + 2$(Ring_hat(i) gt 1);
error(i) = 1;
error(i)$(y_hat(i) = y(i) ) = 0;
errors = sum(tuning(i),error(i));
if( errors0 > errors, C0=C; errors0 = errors );
rate = errors/card(tuning);
Result("C=7","errors") = errors;
Result("C=7","rate") = rate;

C = C0;
*display Result, C0, errors0;
display Result, C;




,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),OptimalLocal (2),1534.0031,21,3011,QCP,CONOPT,0
1,Normal (1),OptimalLocal (2),798.6040,3001,3012,QCP,CONOPT,0.156
2,Normal (1),OptimalLocal (2),1534.0031,21,3011,QCP,CONOPT,0.016
3,Normal (1),OptimalLocal (2),1534.0031,3001,3012,QCP,CONOPT,0.156
4,Normal (1),OptimalLocal (2),1534.0031,21,3011,QCP,CONOPT,0.016
5,Normal (1),OptimalLocal (2),2975.6893,3001,3012,QCP,CONOPT,0.11
6,Normal (1),OptimalLocal (2),1534.0031,21,3011,QCP,CONOPT,0
7,Normal (1),OptimalLocal (2),4406.6884,3001,3012,QCP,CONOPT,0.094
8,Normal (1),OptimalLocal (2),1534.0031,21,3011,QCP,CONOPT,0.015
9,Normal (1),OptimalLocal (2),5832.8683,3001,3012,QCP,CONOPT,0.078


In [7]:
%gams_pull -d C
display(C)

,value
0,6.0


Now with the fixed value of C from the above, determine the error rate that you would expect to make on unseen data (this is the first time you should use the test data, and you should not solve any optimization problem with that data, just see if your prediction is correct or not).

In [8]:
%%gams
*# write gams code to simply evaluate the errorrate in your prediction on the test set.

solve Dual max ObjDual using QCP;
psi.L(i)$tuning(i) = ConstraintDual.M(i);
solve Primal min ObjPrimal using QCP;

Ring_hat(i)$test(i) = sum(x(headr), w.L(headr)*Data(i,headr)) + gamma.L ;
y_hat(i)$test(i) = -1 + 2$(Ring_hat(i) gt 1);
error(i) = 1;
error(i)$(y_hat(i) = y(i) ) = 0;
errors = sum(test(i), error(i) );

parameter errorrate;
errorrate = errors/card(test);
display errorrate;


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),OptimalLocal (2),1534.0031,21,3011,QCP,CONOPT,0.015
1,Normal (1),OptimalLocal (2),8678.9388,3001,3012,QCP,CONOPT,0.141


In [9]:
%gams_pull -d errorrate
display(errorrate)

,value
0,0.249631
